In [1]:
import pandas as pd
import sys
import time

sys.path.append('/home/ryfran/PairProphet/pairpro')

import user_blast as ub

In [10]:
s100 = pd.read_csv('/mnt/s/OMA/samples/oma_sample_100.csv')
s100.head()

,protein1_uniprot_id,protein2_uniprot_id,protein1_sequence,protein2_sequence
0,B4SFM3,NaN,MRLLIIEDEPGIAGFLKDGLEEEYFAVDIAYDGKSGLDMALINEYD...,MTELPIDENTPRILIVEDEPKLGQLLIDYLRAAGYAPSLISHGDQV...
1,F8JXB8,Q48T67,MGGLPPVFIEFLGRSTGVKTAMRDVKAELGTTAAEGETSFTKFGRL...,MGESYSVEAVLTAVDKTFGKTLQSAIRSIEGLEKRSTGFSSVSQKA...
2,A0A0J3YAP3,A0A0J3YAP3,MSEMVDTTQTTTEKKLTQSDIRGVFLRSNLFQGSWNFERMQALGFC...,MSEMVDTTQTTTEKKLTQSDIRGVFLRSNLFQGSWNFERMQALGFC...
3,F6EJL2,D1B192,MKAGMTSSPHMSEYRSSILGYPRIGPRRELKRALESYWRGGISQEE...,MLKTYVTGFPRIGEKRELKFALESYWAGKSELSTLQDVAKELRARH...
4,H6RIS0,B1JFQ1,MFGTFLLSAAGDELARVALTVLVYDRTSSPLLAALTFAISHLPWLV...,MLSVLKNRTYRHLFIAQVIALIGTGLATVALGLLAYNLAGAQAGAV...


In [8]:
df, con = ub.make_blast_df(s100[['protein1_sequence', 'protein2_sequence']], path='/mnt/s/OMA/samples/oma_sample_100.db')

/home/ryfran/PairProphet/pairpro/user_blast.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={original_cols[0]: 'query', original_cols[1]: 'subject'},
/home/ryfran/PairProphet/pairpro/user_blast.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['query_id'] = [qid_dict[i] for i in df.iloc[:, 0]]


In [11]:
df[['protein1_alphafold_id', 'protein2_alphafold_id']] = s100[['protein1_uniprot_id', 'protein2_uniprot_id']]

In [12]:
df.head()

,query,subject,query_id,subject_id,pair_id,local_gap_compressed_percent_id,scaled_local_query_percent_id,scaled_local_symmetric_percent_id,query_align_len,query_align_cov,subject_align_len,subject_align_cov,bit_score,protein1_alphafold_id,protein2_alphafold_id
0,MRLLIIEDEPGIAGFLKDGLEEEYFAVDIAYDGKSGLDMALINEYD...,MTELPIDENTPRILIVEDEPKLGQLLIDYLRAAGYAPSLISHGDQV...,125,174,0,0.320833,0.320833,0.329764,240,0.983402,227,0.995851,274.0,B4SFM3,NaN
1,MGGLPPVFIEFLGRSTGVKTAMRDVKAELGTTAAEGETSFTKFGRL...,MGESYSVEAVLTAVDKTFGKTLQSAIRSIEGLEKRSTGFSSVSQKA...,32,38,1,0.147500,0.146160,0.170849,1211,0.971429,861,0.935135,197.0,F8JXB8,Q48T67
2,MSEMVDTTQTTTEKKLTQSDIRGVFLRSNLFQGSWNFERMQALGFC...,MSEMVDTTQTTTEKKLTQSDIRGVFLRSNLFQGSWNFERMQALGFC...,142,151,2,1.000000,1.000000,1.000000,286,1.000000,286,1.000000,1475.0,A0A0J3YAP3,A0A0J3YAP3
3,MKAGMTSSPHMSEYRSSILGYPRIGPRRELKRALESYWRGGISQEE...,MLKTYVTGFPRIGEKRELKFALESYWAGKSELSTLQDVAKELRARH...,55,102,3,0.383798,0.395349,0.386364,731,0.994798,765,0.971391,1311.0,F6EJL2,D1B192
4,MFGTFLLSAAGDELARVALTVLVYDRTSSPLLAALTFAISHLPWLV...,MLSVLKNRTYRHLFIAQVIALIGTGLATVALGLLAYNLAGAQAGAV...,29,107,4,0.241860,0.239631,0.247914,434,0.966443,405,0.970917,200.0,H6RIS0,B1JFQ1


In [14]:
import numpy as np

print(np.sum(df['protein1_alphafold_id'] == df['protein2_alphafold_id']))

1


Some pairs have identical ids! Need to filter

In [16]:
print(np.sum(df['protein1_alphafold_id'].isna())+ np.sum(df['protein2_alphafold_id'].isna()))

24


Many nan in alphafold id column!

In [17]:
df.loc[df['protein1_alphafold_id'] == df['protein2_alphafold_id']]

,query,subject,query_id,subject_id,pair_id,local_gap_compressed_percent_id,scaled_local_query_percent_id,scaled_local_symmetric_percent_id,query_align_len,query_align_cov,subject_align_len,subject_align_cov,bit_score,protein1_alphafold_id,protein2_alphafold_id
2,MSEMVDTTQTTTEKKLTQSDIRGVFLRSNLFQGSWNFERMQALGFC...,MSEMVDTTQTTTEKKLTQSDIRGVFLRSNLFQGSWNFERMQALGFC...,142,151,2,1.0,1.0,1.0,286,1.0,286,1.0,1475.0,A0A0J3YAP3,A0A0J3YAP3


In [19]:
con.execute("""SELECT * FROM protein_pairs LIMIT 1""").df()

,query,subject,query_id,subject_id,pair_id,local_gap_compressed_percent_id,scaled_local_query_percent_id,scaled_local_symmetric_percent_id,query_align_len,query_align_cov,subject_align_len,subject_align_cov,bit_score
0,MRLLIIEDEPGIAGFLKDGLEEEYFAVDIAYDGKSGLDMALINEYD...,MTELPIDENTPRILIVEDEPKLGQLLIDYLRAAGYAPSLISHGDQV...,125,174,0,0.320833,0.320833,0.329764,240,0.983402,227,0.995851,274.0
